In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list 

ref                                                           title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
meirnizri/covid19-dataset                                     COVID-19 Dataset                                     5MB  2022-11-13 15:47:17          15678        441  1.0              
devrimtuner/list-of-moststreamed-songs-on-spotify             Top 100 Spotify Songs👑🎤🎧🎼                            3KB  2022-12-30 05:42:54            604         39  1.0              
thedevastator/analyzing-credit-card-spending-habits-in-india  Credit Card Spending Habits in India               319KB  2022-12-14 07:30:37           1869         62  1.0              
die9origephit/fifa-world-cup-2022-complete-dataset            Fifa World Cu

In [ ]:
!kaggle datasets download -d samuelcortinhas/cats-and-dogs-image-classification

 74% 48.0M/64.4M [00:00<00:00, 258MB/s]
100% 64.4M/64.4M [00:00<00:00, 259MB/s]


In [ ]:
!unzip "/content/cats-and-dogs-image-classification.zip" -d "/content/drive/MyDrive/Kaggle Datasets"

In [ ]:
train_dir = "/content/drive/MyDrive/Kaggle Datasets/Cat Dog/train"
test_dir = "/content/drive/MyDrive/Kaggle Datasets/Cat Dog/test"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
train_generator = train_datagen.flow_from_directory(
    directory=train_dir, target_size=(100, 100), color_mode='rgb', class_mode='binary', batch_size=32, 
    shuffle=True, seed=2022)

Found 549 images belonging to 2 classes.


In [ ]:
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )
validation_generator = test_datagen.flow_from_directory(
    directory=test_dir, target_size=(100, 100), color_mode='rgb', class_mode='binary', 
    batch_size=32, shuffle=True, seed=2022)

Found 134 images belonging to 2 classes.


In [ ]:
import tensorflow as tf

In [ ]:
tf.random.set_seed(2022)
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2,1),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,1),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(1024, activation='relu'), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation='sigmoid')  
])
from tensorflow.keras.callbacks import EarlyStopping
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5,verbose=3,restore_best_weights=True)
model.compile(optimizer='adam',loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(train_generator, validation_data=validation_generator, epochs=50,
                    verbose=2, callbacks=[monitor])

### Transfer learning

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.utils import plot_model

In [12]:
pre_trained = InceptionV3(include_top=False, weights='imagenet', classes=2,
                          pooling='max',
                          classifier_activation='sigmoid')

In [ ]:
plot_model(pre_trained)

In [ ]:
pre_trained.summary()

Freeze the Original weights

In [13]:
pre_trained.trainable = False

Customize the model

In [14]:
model = tf.keras.Sequential([
    pre_trained,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'), 
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(200, activation='relu'), 
    tf.keras.layers.Dense(1, activation='sigmoid')])
from tensorflow.keras.callbacks import EarlyStopping
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5,verbose=3,restore_best_weights=True)
model.compile(optimizer='adam',loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(train_generator, validation_data=validation_generator, epochs=50,
                    verbose=2, callbacks=[monitor])

Epoch 1/50
